In [8]:
import dspy
from aime_dataset import init_dataset

train_set, val_set, test_set = init_dataset()

lm = dspy.LM("openai/gpt-4.1-mini", temperature=1, max_tokens=32000)
dspy.configure(lm=lm)

In [9]:
class GenerateResponse(dspy.Signature):
    """Solve the problem and provide the answer in the correct format."""
    problem = dspy.InputField()
    answer = dspy.OutputField()

program = dspy.ChainOfThought(GenerateResponse)

In [10]:
def metric(example, prediction, trace=None, pred_name=None, pred_trace=None):
    correct_answer = int(example['answer'])
    try:
        llm_answer = int(prediction.answer)
    except ValueError as e:
        return 0
    return int(correct_answer == llm_answer)

In [11]:
evaluate = dspy.Evaluate(
    devset=test_set,
    metric=metric,
    num_threads=32,
    display_table=True,
    display_progress=True
)

evaluate(program)

Average Metric: 75.00 / 150 (50.0%): 100%|██████████| 150/150 [00:00<00:00, 4450.88it/s]

2025/10/31 16:03:23 INFO dspy.evaluate.evaluate: Average Metric: 75 / 150 (50.0%)


,problem,example_answer,reasoning,pred_answer,metric
0,Find the sum of all integer bases $b>9$ for which $17_b$ is a divi...,70,"First, let's understand the problem carefully: We are given number...",70,✔️ [1]
1,"On $\triangle ABC$ points $A, D, E$, and $B$ lie in that order on ...",588,We are given a triangle \( \triangle ABC \) with specified points ...,588,✔️ [1]
2,The 9 members of a baseball team went to an ice-cream parlor after...,16,"There are 9 players and 3 flavors: chocolate (C), vanilla (V), and...",16,✔️ [1]
3,"Find the number of ordered pairs $(x,y)$, where both $x$ and $y$ a...",117,"We are given the equation: \[12x^2 - xy - 6y^2 = 0\] with \(x,y \i...",117,✔️ [1]
4,There are $8!= 40320$ eight-digit positive integers that use each ...,279,We are looking at all 8-digit numbers formed by using each digit f...,279,✔️ [1]
...,...,...,...,...,...
145,Let $S$ be the set of vertices of a regular $24$-gon. Find the num...,113,"We have a regular 24-gon with vertices labeled \(0, 1, 2, \ldots, ...",112,✔️ [0]
146,Let $A_1 A_2 A_3 \ldots A_{11}$ be an $11$-sided non-convex simple...,19,We are given an 11-sided simple polygon \( A_1 A_2 \dots A_{11} \)...,19,✔️ [1]
147,"Let $x_1, x_2, x_3, \ldots$ be a sequence of rational numbers defi...",248,We start with the sequence defined by: \[ x_1 = \frac{25}{11} \] a...,36,✔️ [0]
148,Let $\triangle ABC$ be a right triangle with $\angle A = 90^\circ$...,104,We have a right triangle \(\triangle ABC\) with \(\angle A = 90^\c...,104,✔️ [1]


EvaluationResult(score=50.0, results=<list of 150 results>)

In [12]:
def metric_with_feedback(example, prediction, trace=None, pred_name=None, pred_trace=None):
    correct_answer = int(example['answer'])
    written_solution = example.get('solution', '')
    try:
        llm_answer = int(prediction.answer)
    except ValueError as e:
        feedback_text = f"The final answer must be a valid integer and nothing else. You responded with '{prediction.answer}', which couldn't be parsed as a python integer. Please ensure your answer is a valid integer without any additional text or formatting."
        feedback_text += f" The correct answer is '{correct_answer}'."
        if written_solution:
            feedback_text += f" Here's the full step-by-step solution:\n{written_solution}\n\nThink about what takeaways you can learn from this solution to improve your future answers and approach to similar problems and ensure your final answer is a valid integer."
        return dspy.Prediction(score=0, feedback=feedback_text)

    score = int(correct_answer == llm_answer)

    feedback_text = ""
    if score == 1:
        feedback_text = f"Your answer is correct. The correct answer is '{correct_answer}'."
    else:
        feedback_text = f"Your answer is incorrect. The correct answer is '{correct_answer}'."
    
    if written_solution:
        feedback_text += f" Here's the full step-by-step solution:\n{written_solution}\n\nThink about what takeaways you can learn from this solution to improve your future answers and approach to similar problems."

    return dspy.Prediction(score=score, feedback=feedback_text)

In [13]:
from dspy import GEPA

optimizer = GEPA(
    metric=metric_with_feedback,
    auto="light",
    num_threads=32,
    track_stats=True,
    reflection_minibatch_size=3,
    reflection_lm=dspy.LM(model="openai/gpt-5", temperature=1.0, max_tokens=32000)
)

optimized_program = optimizer.compile(
    program,
    trainset=train_set,
    valset=val_set,
)

2025/10/31 16:03:23 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 560 metric calls of the program. This amounts to 6.22 full evals on the train+val set.
2025/10/31 16:03:23 INFO dspy.teleprompt.gepa.gepa: Using 45 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget. GEPA requires you to provide the smallest valset that is just large enough to match your downstream task distribution, while providing as large trainset as possible.
GEPA Optimization:   0%|          | 0/560 [00:00<?, ?rollouts/s]2025/10/31 16:03:23 INFO dspy.evaluate.evaluate: Average Metric: 20.0 / 45 (44.4%)
2025/10/31 16:03:23 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.4444444444444444
GEPA Optimization:   8%|▊         | 45/560 [00:00<00:01, 412.41rollouts/s]2025/10/31 16:03:23 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.4444444444444444


Average Metric: 2.00 / 3 (66.7%): 100%|██████████| 3/3 [00:00<00:00, 2431.01it/s]

2025/10/31 16:03:23 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)
2025/10/31 16:03:23 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for predict: You are given a single “problem” statement and must solve it and output the result in the required format.

Output format
- Provide two sections in this order:
  - reasoning: a concise derivation showing the key steps, identities, or checks you used.
  - answer: a single line containing only the final result (number, expression, or object exactly as requested). Do not include extra words, symbols, or explanations in the answer line.
- Keep reasoning clear and minimal; avoid unnecessary brute force.

General problem-solving guidance (use what applies)
1) Base and digit constraints
- If a number in base 10 equals a representation in another base, equate their value expansions directly.
  - Example: decimal abc is 100a + 10b + c. Base-9 bca is 81b + 9c + a.
- Enforce base-digit validity: in base k, each digit must 

2025/10/31 16:03:23 INFO dspy.evaluate.evaluate: Average Metric: 3.0 / 3 (100.0%)
2025/10/31 16:03:23 INFO dspy.teleprompt.gepa.gepa: Iteration 1: New subsample score 3 is better than old score 2. Continue to full eval and add to candidate pool.
2025/10/31 16:03:23 INFO dspy.evaluate.evaluate: Average Metric: 18.0 / 45 (40.0%)
2025/10/31 16:03:23 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Full valset score for new program: 0.4
2025/10/31 16:03:23 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Full train_val score for new program: 0.4
2025/10/31 16:03:23 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Individual valset scores for new program: [0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
2025/10/31 16:03:23 INFO dspy.teleprompt.gepa.gepa: Iteration 1: New valset pareto front scores: [0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,

Average Metric: 1.00 / 3 (33.3%): 100%|██████████| 3/3 [00:00<00:00, 4154.15it/s]

...


Average Metric: 3.00 / 3 (100.0%): 100%|██████████| 3/3 [00:00<00:00, 2976.79it/s]

2025/10/31 16:03:24 INFO dspy.evaluate.evaluate: Average Metric: 3.0 / 3 (100.0%)
2025/10/31 16:03:24 INFO dspy.teleprompt.gepa.gepa: Iteration 12: All subsample scores perfect. Skipping.
2025/10/31 16:03:24 INFO dspy.teleprompt.gepa.gepa: Iteration 12: Reflective mutation did not propose a new candidate
2025/10/31 16:03:24 INFO dspy.teleprompt.gepa.gepa: Iteration 13: Selected program 9 score: 0.5111111111111111



Average Metric: 0.00 / 3 (0.0%): 100%|██████████| 3/3 [00:00<00:00, 4125.54it/s]

2025/10/31 16:03:24 INFO dspy.evaluate.evaluate: Average Metric: 0.0 / 3 (0.0%)


2025/10/31 16:03:24 INFO dspy.teleprompt.gepa.gepa: Iteration 13: Proposed new text for predict: You will be given a single math problem under a "problem" field. Your job is to solve it correctly and output only the final answer in the exact format explicitly requested by the problem. Keep your output concise: unless the prompt explicitly asks for steps or reasoning, return only the final value (for example, an integer, simplified fraction, simplified radical, or required expression). Do not include extra text, labels, or formatting.

Core rules
- Parse exactly what the problem asks you to output (e.g., an integer, p+q for a reduced fraction p/q, m+n for a reduced probability m/n, a simplified radical, etc.).
- Reduce fractions to lowest terms before forming p+q or m+n. Simplify radicals/rational expressions fully.
- Use exact arithmetic; never return decimal approximations unless explicitly requested.
- Output only the final requested value.

Quality checks before finalizing
- Verify 

In [14]:
evaluate(optimized_program)

Average Metric: 60.00 / 150 (40.0%): 100%|██████████| 150/150 [00:00<00:00, 1838.17it/s]

2025/10/31 16:03:24 INFO dspy.evaluate.evaluate: Average Metric: 60 / 150 (40.0%)


,problem,example_answer,reasoning,pred_answer,metric
0,Find the sum of all integer bases $b>9$ for which $17_b$ is a divi...,70,Convert the numbers from base \(b\) to base 10: - \(17_b = 1 \time...,70,✔️ [1]
1,"On $\triangle ABC$ points $A, D, E$, and $B$ lie in that order on ...",588,We are given a triangle \( ABC \) with points on sides \(AB\) and ...,294,✔️ [0]
2,The 9 members of a baseball team went to an ice-cream parlor after...,16,"We have 9 players, and they select flavors from {C, V, S} with the...",16,✔️ [1]
3,"Find the number of ordered pairs $(x,y)$, where both $x$ and $y$ a...",117,Given the equation: \[ 12x^2 - xy - 6y^2 = 0 \] We want to find al...,117,✔️ [1]
4,There are $8!= 40320$ eight-digit positive integers that use each ...,279,We want to find the number of 8-digit integers using each digit 1 ...,279,✔️ [1]
...,...,...,...,...,...
145,Let $S$ be the set of vertices of a regular $24$-gon. Find the num...,113,We are given a regular 24-gon and want to pair up all 24 vertices ...,1,✔️ [0]
146,Let $A_1 A_2 A_3 \ldots A_{11}$ be an $11$-sided non-convex simple...,19,Let \( A_1 = O \) be the origin in the plane. We consider the poin...,143,✔️ [0]
147,"Let $x_1, x_2, x_3, \ldots$ be a sequence of rational numbers defi...",248,"We have the sequence defined by: \[ x_1 = \frac{25}{11}, \quad x_{...",660,✔️ [0]
148,Let $\triangle ABC$ be a right triangle with $\angle A = 90^\circ$...,104,Given a right triangle \( \triangle ABC \) with \(\angle A = 90^\c...,114,✔️ [0]


EvaluationResult(score=40.0, results=<list of 150 results>)